In [1]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import inspect
import io 
import json
import numpy as np
import pandas as pd
import requests 
from scipy.optimize import curve_fit

In [2]:
urlData = requests.get('https://corona.lmao.ninja/v2/historical/argentina').content

In [3]:
data = urlData.decode('utf-8')
data

'{"country":"Argentina","province":["mainland"],"timeline":{"cases":{"8/6/20":228195,"8/7/20":235677,"8/8/20":241811,"8/9/20":246499,"8/10/20":253868,"8/11/20":260911,"8/12/20":268574,"8/13/20":276072,"8/14/20":282437,"8/15/20":289100,"8/16/20":294569,"8/17/20":299126,"8/18/20":305966,"8/19/20":312659,"8/20/20":320884,"8/21/20":329043,"8/22/20":336802,"8/23/20":342154,"8/24/20":350867,"8/25/20":359638,"8/26/20":370188,"8/27/20":380292,"8/28/20":392009,"8/29/20":401239,"8/30/20":408426,"8/31/20":417735,"9/1/20":428239,"9/2/20":439172,"9/3/20":451198,"9/4/20":461882},"deaths":{"8/6/20":4251,"8/7/20":4411,"8/8/20":4523,"8/9/20":4606,"8/10/20":4764,"8/11/20":5004,"8/12/20":5213,"8/13/20":5362,"8/14/20":5527,"8/15/20":5637,"8/16/20":5703,"8/17/20":5814,"8/18/20":6048,"8/19/20":6330,"8/20/20":6517,"8/21/20":6730,"8/22/20":6848,"8/23/20":6985,"8/24/20":7366,"8/25/20":7563,"8/26/20":7839,"8/27/20":8050,"8/28/20":8271,"8/29/20":8353,"8/30/20":8457,"8/31/20":8660,"9/1/20":8919,"9/2/20":9118,"9/3

In [4]:
data_load = json.loads(data)
data_load

{'country': 'Argentina',
 'province': ['mainland'],
 'timeline': {'cases': {'8/6/20': 228195,
   '8/7/20': 235677,
   '8/8/20': 241811,
   '8/9/20': 246499,
   '8/10/20': 253868,
   '8/11/20': 260911,
   '8/12/20': 268574,
   '8/13/20': 276072,
   '8/14/20': 282437,
   '8/15/20': 289100,
   '8/16/20': 294569,
   '8/17/20': 299126,
   '8/18/20': 305966,
   '8/19/20': 312659,
   '8/20/20': 320884,
   '8/21/20': 329043,
   '8/22/20': 336802,
   '8/23/20': 342154,
   '8/24/20': 350867,
   '8/25/20': 359638,
   '8/26/20': 370188,
   '8/27/20': 380292,
   '8/28/20': 392009,
   '8/29/20': 401239,
   '8/30/20': 408426,
   '8/31/20': 417735,
   '9/1/20': 428239,
   '9/2/20': 439172,
   '9/3/20': 451198,
   '9/4/20': 461882},
  'deaths': {'8/6/20': 4251,
   '8/7/20': 4411,
   '8/8/20': 4523,
   '8/9/20': 4606,
   '8/10/20': 4764,
   '8/11/20': 5004,
   '8/12/20': 5213,
   '8/13/20': 5362,
   '8/14/20': 5527,
   '8/15/20': 5637,
   '8/16/20': 5703,
   '8/17/20': 5814,
   '8/18/20': 6048,
   '8/19

In [5]:
df_timeline = pd.json_normalize(data_load['timeline']['cases']).T
df_deaths = pd.json_normalize(data_load['timeline']['deaths']).T
df_recovered = pd.json_normalize(data_load['timeline']['recovered']).T

In [6]:
df_timeline.columns = ['confirmed']
df_deaths.columns = ['deaths']
df_recovered.columns = ['recovered']

In [7]:
data_argentina = pd.concat([df_timeline, df_deaths, df_recovered], axis=1, sort=False)
data_argentina

,confirmed,deaths,recovered
8/6/20,228195,4251,99852
8/7/20,235677,4411,103297
8/8/20,241811,4523,108242
8/9/20,246499,4606,108242
8/10/20,253868,4764,108242
8/11/20,260911,5004,181389
8/12/20,268574,5213,187283
8/13/20,276072,5362,192434
8/14/20,282437,5527,199005
8/15/20,289100,5637,205697


In [8]:
data_argentina.index=pd.to_datetime(data_argentina.index.astype(str), format='%m/%d/%y')

In [9]:
data_argentina['num_id'] = np.arange(len(data_argentina) + 1)[1:]

In [10]:
data_argentina

,confirmed,deaths,recovered,num_id
2020-08-06,228195,4251,99852,1
2020-08-07,235677,4411,103297,2
2020-08-08,241811,4523,108242,3
2020-08-09,246499,4606,108242,4
2020-08-10,253868,4764,108242,5
2020-08-11,260911,5004,181389,6
2020-08-12,268574,5213,187283,7
2020-08-13,276072,5362,192434,8
2020-08-14,282437,5527,199005,9
2020-08-15,289100,5637,205697,10


In [11]:
predicted_days = 7
firstPredictionDay = data_argentina.index[-1] + datetime.timedelta(days = 1)

In [12]:
class Models:    
    def cuadratic_model(x, a, b, c):
        return (a*x**2 + b*x +c)
    
    def cubic_model(x, a, b, c, d):
        return (a*x**3 + b*x**2 +c*x + d)
    
    def fourth_pow_model(x, a, b, c, d, e):
        return (a*x**4 + b*x**3 +c*x**2 + d*x + e)
    
    def fiveth_pow_model(x, a, b, c, d, e, f):
        return (a*x**5 + b*x**4 +c*x**3 + d*x**2 + e*x + f)
    
    def sixh_pow_model(x, a, b, c, d, e, f, g):
        return (a*x**6 + b*x**5 +c*x**4 + d*x**3 + e*x**2 + f*x + g)

In [13]:
def formatPrediction(firstPredictionDay, day, prediction):
    formatedDate = str(firstPredictionDay + datetime.timedelta(days = day))[0:10]
    formatedPrediction = str(int(prediction))
    return "|" + formatedDate + " | " + formatedPrediction + "|"

def printPrediction(model, popt, x, y):
    firstPredictionDayId = x.max() + 1
    print("--- Predictions ---")
    for day in range (0, predicted_days):
        predictionId = firstPredictionDayId + day
        predictionValue = model(predictionId, *popt)
        print(formatPrediction(firstPredictionDay, day, predictionValue))
    print("-------------------")

In [14]:
def printFuncionParameters(model, popt):
        print('\n', model.__name__)
        args = inspect.getfullargspec(model).args[1:]
        for arg, value in zip(args, popt):
            print( arg, " = ", value)

In [15]:
def predictData(model, x, y):
    popt, _ = curve_fit(model, x, y)
    printPrediction(model, popt, x, y)
    printFuncionParameters(model, popt)
    print()

In [16]:
def isModel(methodName):
    return not(methodName.startswith('__'))

for methodName in dir(Models):
    if(isModel(methodName)):
        print(methodName)
        predictData(getattr(Models, methodName), data_argentina.num_id, data_argentina.confirmed)

cuadratic_model
--- Predictions ---
|2020-09-05 | 470776|
|2020-09-06 | 481758|
|2020-09-07 | 492934|
|2020-09-08 | 504306|
|2020-09-09 | 515873|
|2020-09-10 | 527635|
|2020-09-11 | 539592|
-------------------

 cuadratic_model
a  =  97.60013273766373
b  =  4832.417243583772
c  =  227178.29088996458

cubic_model
--- Predictions ---
|2020-09-05 | 475510|
|2020-09-06 | 488323|
|2020-09-07 | 501618|
|2020-09-08 | 515412|
|2020-09-09 | 529722|
|2020-09-10 | 544565|
|2020-09-11 | 559960|
-------------------

 cubic_model
a  =  2.89175184049538
b  =  -36.86632948488114
c  =  6527.273046750172
d  =  222445.07122836303

fiveth_pow_model
--- Predictions ---
|2020-09-05 | 470477|
|2020-09-06 | 479474|
|2020-09-07 | 487612|
|2020-09-08 | 494642|
|2020-09-09 | 500289|
|2020-09-10 | 504245|
|2020-09-11 | 506172|
-------------------

 fiveth_pow_model
a  =  -0.011430648043665448
b  =  0.6805110450252038
c  =  -9.00236528124761
d  =  0.3317433661983359
e  =  6973.716646736724
f  =  220946.7361631837
